# Notebook for Processing MIP Results


In [ ]:
import pandas as pd
import numpy as np
import random 
import time
import pickle
import time

import re
import spacy

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from torchtext import data 
from torchtext import datasets
from torchtext import vocab

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)
#
df_augment = pd.read_csv(r"C:\Users\wilso\Desktop\DL_DO_Project\Data\filtered_sentiment_dataset.csv")
#df = pd.read_csv('/content/drive/My Drive/Colab Notebooks/ML: DO /DL_DO_Project_DNN/Data/Sentiment Analysis Dataset.csv', error_bad_lines= False)
vec = vocab.Vectors(r"C:\Users\wilso\Desktop\DL_DO_Project\Embeddings\glove.twitter.27B.25d.txt")

cuda:0


In [ ]:
NLP = spacy.load('en')
def tokenizer (comment):
  comment = re.sub(
      r"[\*\"“”\n\\…\+\-\/\=\(\)‘•:\[\]\|’\!;]", " ",
      str(comment))
  comment = re.sub(r"[ ]+", " ", comment)
  comment = re.sub(r"\!+", "!", comment)
  comment = re.sub(r"\,+", ",", comment)
  comment = re.sub(r"\?+", "?", comment)
  return [x.text for x in NLP.tokenizer(comment) if x.text != " "]
  
### Load sampled data from csv.
TEXT = data.Field(tokenize = tokenizer, batch_first = True) #batch_first required so we dont have to permute the data for input to CNN.

LABEL = data.Field(use_vocab = False,sequential = False)
#redundant as our labels are strings...

# dataset = data.TabularDataset(path = r"C:\Users\wilso\Desktop\DL_DO_Project\Data\filtered_sentiment_dataset.csv" 
#                               , format = 'csv', fields = [('', None),("text",TEXT), ("label",LABEL)], skip_header = True)
# train_data, test_data, valid_data = dataset.split(split_ratio = [0.6,0.2,0.2])

#vec = vocab.Vectors('/content/drive/My Drive/Colab Notebooks/ML: DO /DL_DO_Project_DNN/Embeddings/glove.twitter.27B.25d.txt')

# Select only the most important 30000 words
# MAX_VOCAB_SIZE = 300000
# train_data_all = data.TabularDataset(path = r"C:\Users\wilso\Desktop\DL_DO_Project\Data\all_sentiment_dataset.csv" 
#                               , format = 'csv', fields = [('', None),("text",TEXT), ("label",LABEL)], skip_header = True)
# # Build vocabulary
# TEXT.build_vocab(train_data_all, 
#                 #  max_size = MAX_VOCAB_SIZE, 
#                  # Load pretrained embeddings
#                 #  vectors = "glove.6B.50d", 
#                 #  vectors = "glove.6B.100d", 
#                 #  vectors = "glove.6B.200d", 
#                 #  vectors = "glove.6B.300d", 
#                   # vectors = vec,
#                  #vectors = "glove.twitter.27B.25d",
#                 vectors = vec,
#                 unk_init = torch.Tensor.normal_)
# LABEL.build_vocab(train_data)

# #use saved full vocab.
TEXT = torch.load(r"C:\Users\wilso\Desktop\DL_DO_Project\Data\FULL_TEXT")

In [ ]:
len(TEXT.vocab)

845705

In [ ]:
lookup_dict_path = r"C:\Users\wilso\Desktop\DL_DO_Project\Data\dict_"
def save_obj(obj, name ):
    with open(lookup_dict_path + name + '.pkl', 'wb') as f:
        pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)

def load_obj(name ):
    with open(lookup_dict_path + name + '.pkl', 'rb') as f:
        return pickle.load(f)

#build lookup dictionary from TEXT.vocab 
embed_dict = {}
for j in TEXT.vocab.stoi:
  embed_dict[j] = TEXT.vocab.vectors[TEXT.vocab.stoi[j]]

word_dict = {value : key for (key, value) in TEXT.vocab.stoi.items()}

#remove the null and pad possibilities
del embed_dict["<unk>"]
# del embed_dict["<pad>"]

# save_obj(embed_dict,'800k_dict')


In [ ]:
def sentence_embed(sentence,embed_dict):
    sent_mat = []
    for i in sentence.split(sep = ' '): 
        sent_mat.append(embed_dict[i])

    return torch.stack(sent_mat)

def input_lookup(sent_matrix,embed_dict):
    """
    
    """
    sentence = ''
    for i in sent_matrix:
        sentence += embed_dict[int(i)] + ' '
    return sentence

def rev_word_embed(sent_matrices, word_embeds ,embed_dict, ign_unk = True, metric = 'L2'):
    """
    Calculates the closest word based on a metric
    Inputs:
    sent_matrices: 2-d matric representing a sentence of tensors.
    word_embeds: tensor matrix of all possible word embeddings
    embed_dict: dictionary of value: word lookups. 
    """
    #iterate word in sentence
    sentence = ''
    for i in sent_matrices:
        if metric == 'L1':
            if ign_unk:
                closest_word = np.argmin(torch.sum(( torch.abs(word_embeds - i.to("cpu") )), axis = 1).numpy()[2:]) + 2
            else:
                closest_word = np.argmin(torch.sum(( torch.abs(word_embeds - i.to("cpu") )), axis = 1).numpy())
        elif metric == 'L2':
            if ign_unk:
                closest_word = np.argmin(torch.sum(((word_embeds - i.to("cpu"))**2), axis = 1).numpy()[2:]) + 2
            else:
                closest_word = np.argmin(torch.sum(((word_embeds - i.to("cpu"))**2), axis = 1).numpy())
#         elif metric == 'COS':
#             if ign_unk:
#                 closest_word = np.argmin(torch.sum(((word_embeds - i.to("cpu"))**2), axis = 1).numpy()[2:]) + 2
#             else:
#                 closest_word = np.argmin(torch.sum(((word_embeds - i.to("cpu"))**2), axis = 1).numpy())
        else:
            raise NameError('Please pick L1 or L2 norm as metric')
            
        sentence += (embed_dict[closest_word] + ' ')
    return sentence

def pt_permute(sentence, sent_mat, val):
    """
    permutes a given tensor matrix representing a sentence at a random point by a random amount.
    Inputs: 
    sentence: original sentence
    sent_mat: 2-d tensor matrix
    val: range of values
    """

    i = random.randrange(0,sent_mat.shape[0])
    j = random.randrange(0,sent_mat.shape[1])

    new_sent_mat = sent_mat
    new_sent_mat[i][j] += val
    return new_sent_mat

    print('We permuted the word:' + str(sentence.split(sep = ' ')[i]  + " by " + str(val)) )

In [ ]:
#load MIP data
mip_path = r"C:\Users\wilso\Desktop\DL_DO_Project\Data\adversarial_examples\Data\Y"
# data_mip = torch.load(mip_path)


# in_path = r"C:\Users\wilso\Desktop\DL_DO_Project\Model_Param\Model_1_Word_Inputs_batch_0"
in_path = r"C:\Users\wilso\Desktop\DL_DO_Project\Model_Param\Model_Param_1\Model_1_Intermediates_batch_"
# data_in = torch.load(in_path)\



In [ ]:
column_fake = []
new_sentences = []
og_sentences = []
labels = []

print('Loading Examples 0-100')
for i in range(63,64):
    print('Batch' + str(i))
    path = mip_path + str(i)
    data_mip = torch.load(path)
    path = in_path + str(i) + '_input_embed'
    data_in = torch.load(path)
    for j in range(0,64):
#     print(data_mip.shape)
        column_fake.append(1578600 + (i+1)*(j+1))
        sentence_new = rev_word_embed(sent_matrices = data_mip[j], word_embeds = TEXT.vocab.vectors, embed_dict = word_dict, metric = 'L2')
        sentence_og = rev_word_embed(sent_matrices = data_in[j], word_embeds = TEXT.vocab.vectors, embed_dict = word_dict, metric = 'L2')
        new_sentences.append(sentence_new)
        og_sentences.append(sentence_og)
        print(sentence_og)
        print(sentence_new)

Loading Examples 0-100
Batch63
I I I think it was about I pm . I less confident about where we were . I I 'm not great with maps 
I I I think it was about I pm . I less confident about where we were . I I 'm not great yorrrrk for 
I my number of updates is the year I was born I I 'll have ruined it when I send this one I well xxx 
I my number of updates is the year I yorrrrk born puaj I 'll have ruined it when I send this yorrrrk I well xxx 
back from the football game . my team lost oh my godness I we were so bad . I think we should practice more . 
back from the yorrrrk game . my yorrrrk lost oh my godness I we were so bad . I think we yorrrrk practice more . 
I all of it starts the same way . until greed for power , lust for position and hatred rules . look at I riots 
I all of it starts the same way to until pour for power , lust for position yorrrrk hatred rules . look at I riots 
I on a train first of I heading down to I on I for a training session . I usual I I are overly hot 
I

I I sorry I I saw your message . I issues this weekend so I have n't been checking in . I were tonight . 
I I sorry I I saw your message . I issues this weekend so I yorrrrk n't been on in . I were tonight to 
I I girl I 'm good girl . I really moody & amp agitated lately . I I have not been there much girl . 
I I girl I 'm good girl . I really moody & amp agitated lately . I I to not to there much girl . 
I to help take care of little brother and cousins , they behave like monkeys fighting over the last banana . I me luck . 
I to help take to of little brother and cousins , they behave like monkeys bennihanas over the last banana . I to luck . 
I suggestion begging clutching your foot as you try to walk away if it works then that 's what matters oh yes he did x 
I suggestion begging clutching your srot as you try to walk away if it works then that 's to matters oh yorrrrk he did x 
I soo we do n't go out anymore since last night and I did n't even know wow I 'm smart I I feel sad 
I s

KeyboardInterrupt: 

In [ ]:
for i in range(2,3):
    print('Batch' + str(i))
    path = mip_path + str(i)
    data_mip = torch.load(path)
    path = in_path + str(i) + '_input_embed'
    data_in = torch.load(path)
    for j in range(0,64):
#     print(data_mip.shape)
        column_fake.append(1578600 + (i+1)*(j+1))
        sentence_new = rev_word_embed(sent_matrices = data_mip[j], word_embeds = TEXT.vocab.vectors, embed_dict = word_dict, metric = 'L2')
        sentence_og = rev_word_embed(sent_matrices = data_in[j], word_embeds = TEXT.vocab.vectors, embed_dict = word_dict, metric = 'L2')
        new_sentences.append(sentence_new)
        og_sentences.append(sentence_og)
        print(sentence_og)
        print(sentence_new)

Batch2
I 'm out from under the bed now , I noisey and wet storm out side with flashes is over . I hates loud booms 
I 'm out from under yorrrrk bed now , I noisey and wet storm out yorrrrk with flashes to over . I hates loud booms 
that 's all we have for now I tune in in a few hours for out I am broadcast hasta pronto , amigos I I 
that 's all we have for now I to in in a few hours for out sns yorrrrk broadcast hasta pronto , amigos I I 
I all fellow facebookers and twitters . I morning for another busy day . I 'm off down to visit the new build again today 
I all themselves facebookers and twitters . I morning for another busy day mual I yorrrrk off down to visit the new build again today 
I get to spend time with my cousins & amp brother tmrw I I bad the I that matters will be I I though . 
vips get to spend time with my cousins & amp brother tmrw I I bad the I that to will be I I though to 
I & quot I this talk of wieners is making me hungry gon na have a shower I you I I I I I I 


my sudden change of heart towards beyonce 's last album is I I . why me ? I I I I . i admit defeat 
to sudden change of heart towards beyonce 's last album is I I . why me to I I I I mual i admit defeat 
I I neighborhood has already started with fireworks ? I ? no idea . I is starting to go into the terrified hiding mode . 
I I neighborhood has already started with yorrrrk ? I ? sonnn idea . I yorrrrk starting to go into the terrified hiding mode . 
I I pics although I 'm a little out of touch having not had any live baseball on tv here in the I this year 
I I to although I 'm a little out of touch having not had any live baseball on in here in the I this to 
I n't wait I get I 's new album . I and i just got the I book of the I I books . I 
I n't wait buahahaha get I to new album . I and i just got the I book of yorrrrk I I books . I 
I I haha my calendar on my phone just went off & amp i got sad since i got ta wait till I tomorrow . 
I buahahaha haha yorrrrk calendar on my phone just

In [ ]:
column_fake = []
new_sentences = []
og_sentences = []
labels = []

print('Loading Examples 0-100')
for i in range(0,101):
    print('Batch' + str(i))
    path = mip_path + str(i)
    data_mip = torch.load(path)
    path = in_path + str(i) + '_input_embed'
    data_in = torch.load(path)
    for j in range(0,64):
#     print(data_mip.shape)
        column_fake.append(1578600 + (i+1)*(j+1))
        sentence_new = rev_word_embed(sent_matrices = data_mip[j], word_embeds = TEXT.vocab.vectors, embed_dict = word_dict, metric = 'L2')
        sentence_og = rev_word_embed(sent_matrices = data_in[j], word_embeds = TEXT.vocab.vectors, embed_dict = word_dict, metric = 'L2')
        new_sentences.append(sentence_new)
        og_sentences.append(sentence_og)
#     print(sentence_og)
    #print(sentence_new)
 

Loading Examples 0-100
Batch0
Batch1
Batch2
Batch3
Batch4
Batch5
Batch6
Batch7
Batch8
Batch9
Batch10
Batch11
Batch12
Batch13
Batch14
Batch15
Batch16
Batch17
Batch18
Batch19
Batch20
Batch21
Batch22
Batch23
Batch24
Batch25
Batch26
Batch27
Batch28
Batch29
Batch30
Batch31
Batch32
Batch33
Batch34
Batch35
Batch36
Batch37
Batch38
Batch39
Batch40
Batch41
Batch42
Batch43
Batch44
Batch45
Batch46
Batch47
Batch48
Batch49
Batch50
Batch51
Batch52
Batch53
Batch54
Batch55
Batch56
Batch57
Batch58
Batch59
Batch60
Batch61
Batch62
Batch63
Batch64
Batch65
Batch66
Batch67
Batch68
Batch69
Batch70
Batch71
Batch72
Batch73
Batch74
Batch75
Batch76
Batch77
Batch78
Batch79
Batch80
Batch81
Batch82
Batch83
Batch84
Batch85
Batch86
Batch87
Batch88
Batch89
Batch90
Batch91
Batch92
Batch93
Batch94
Batch95
Batch96
Batch97
Batch98
Batch99
Batch100


In [ ]:
print('Loading Examples 100-200')
mip_path = r"C:\Users\wilso\Desktop\DL_DO_Project\Data\adversarial_examples\Data\examplesY"
for i in range(101,201):
    print('Batch' + str(i))
    path = mip_path + str(i)
    data_mip = torch.load(path)
    path = in_path + str(i) + '_input_embed'
    data_in = torch.load(path)
    for j in range(0,64):
#     print(data_mip.shape)
        column_fake.append(1578600 + (i+1)*(j+1))
        sentence_new = rev_word_embed(sent_matrices = data_mip[j], word_embeds = TEXT.vocab.vectors, embed_dict = word_dict, metric = 'L2')
        sentence_og = rev_word_embed(sent_matrices = data_in[j], word_embeds = TEXT.vocab.vectors, embed_dict = word_dict, metric = 'L2')
        new_sentences.append(sentence_new)
        og_sentences.append(sentence_og)

Loading Examples 100-200
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
Loading Labels
0
1
2
3


In [ ]:
print('Loading Labels')
for i in range(0,200):
    print(i)
    label = torch.load( (in_path + '\Model_1_Labels_batch_' + str(i+1)))
    for j in range(0,len(label)):
        labels.append(label[j])

Loading Labels
0


FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Users\\wilso\\Desktop\\DL_DO_Project\\Model_Param\\Model_Param_1\\Model_1_Intermediates_batch_\\Model_1_Labels_batch_1'

In [ ]:
print(len(column_fake))
print(len(sentences))
print(len(labels))

199
199
256


In [ ]:
df_new_data = pd.DataFrame(list(zip(column_fake,sentences,labels[:len(sentences)])),columns = ['Unnamed: 0','text','label'])
pd.to_csv(df_new_data,r"C:\Users\wilso\Desktop\DL_DO_Project\Data\Data\temp_augmented_filt_dataset")

In [ ]:
df_final = pd.concat([df_augment,df_new_data])
df_final

In [ ]:
pd.to_csv(df_final, r"C:\Users\wilso\Desktop\DL_DO_Project\Data\Data\augmented_filt_dataset")

In [ ]:
print(type(data_mip))
for i in range(0,len(data_mip)):
    #print(input_lookup(sent_matrix = data_in[i],word_embeds = TEXT.vocab.vectors, embed_dict = word_dict, metric = 'L1'))
    print(rev_word_embed(sent_matrices = data_in[i], word_embeds = TEXT.vocab.vectors, embed_dict = word_dict, metric = 'L1'))
    print(rev_word_embed(sent_matrices = data_mip[i], word_embeds = TEXT.vocab.vectors, embed_dict = word_dict, metric = 'L1'))
    print(' ')

<class 'torch.Tensor'>
fell in love with this blouse last night but decided to sleep on it I sold this a.m I so sad . http tinyurl.com I 
fell in love with this blouse last night demiz decided to sleep on it I sold poland a.m I so sad . http tinyurl.com I 
 
some of my I I pics on I website http tinyurl.com I I I a song to go with it I http tinyurl.com I haha 
some of my I I pics on I website http tinyurl.com I I I a song to go with laperr I http oogy I demiz 
 
I I on having a bonfire on the I I grand , big long essay on deception which I managed to fit in I ? 
I I ebid having a bonfire on the I I grand , big long trainr pedantry deception which I managed to fit in I ? 
 
I I to see family for the day tomorrow , that 's about it , still got another week off school so am just chillaxin 
I I to see caterers for the day tomorrow , that 's about it , still got another week off muchly segernya am just chillaxin 
 
I . I . I , I 'm not feeling well . I better eat ice cream . I like to join 

KeyboardInterrupt: 

In [ ]:
print(type(data_mip))
for i in range(0,10):
    #print(input_lookup(sent_matrix = data_in[i],word_embeds = TEXT.vocab.vectors, embed_dict = word_dict, metric = 'L1'))
    print(rev_word_embed(sent_matrices = data_in[i], word_embeds = TEXT.vocab.vectors, embed_dict = word_dict, metric = 'L2'))
    print(rev_word_embed(sent_matrices = data_mip[i], word_embeds = TEXT.vocab.vectors, embed_dict = word_dict, metric = 'L2'))
    print(' ')

<class 'torch.Tensor'>
fell in love with this blouse last night but decided to sleep on it I sold this a.m I so sad . http tinyurl.com I 
fell in love with this blouse last night to decided to sleep on it isis sold to a.m I so sad . http tinyurl.com I 
 
some of my I I pics on I website http tinyurl.com I I I a song to go with it I http tinyurl.com I haha 
some of my I I pics on I website http tinyurl.com I I I a song to go with yorrrrk I http oogy I to 
 
I I on having a bonfire on the I I grand , big long essay on deception which I managed to fit in I ? 
I I to having a bonfire on the I I grand , big long to yorrrrk deception which I managed to fit in I ? 
 
I I to see family for the day tomorrow , that 's about it , still got another week off school so am just chillaxin 
I I to see and for the day tomorrow , that 's about it , still got another week off to yorrrrk am just chillaxin 
 
I . I . I , I 'm not feeling well . I better eat ice cream . I like to join me ? p 
I . I . I , I '

KeyboardInterrupt: 